In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import re
from collections import Counter
from textblob import TextBlob

In [ ]:
def words(text): return re.findall(r'\w+', text.lower())

WORDS = Counter(words(open('/content/drive/MyDrive/graduate/nlp/week1/big.txt').read()))
letters    = 'abcdefghijklmnopqrstuvwxyz'
def P(word, N=sum(WORDS.values())): 
    "Probability of `word`."
    return WORDS[word] / N

def correction(word): 
    "Most probable spelling correction for word."
    return max(candidates(word), key=P)

def candidates(word): 
    "Generate possible spelling corrections for word."
    if known(inserts1(word)):
        return known(inserts1(word))
    elif known(transposes(word)):
        return known(transposes(word))
    elif known(replaces(word)):
        return known(replaces(word))
    elif known(inserts2(word)):
        return known(inserts2(word))
    else:
        return (known([word]) or known(edits1(word)) or known(edits2(word)) or [word])


def known(words): 
    "The subset of `words` that appear in the dictionary of WORDS."
    return set(w for w in words if w in WORDS)

def edits1(word):
    "All edits that are one edit away from `word`."
    splits     = [(word[:i], word[i:]) for i in range(len(word) + 1)]
    deletes    = [L + R[1:] for L, R in splits if R]
    transposes = [L + R[1] + R[0] + R[2:] for L, R in splits if len(R)>1]
    replaces   = [L + c + R[1:] for L, R in splits if R for c in letters]
    inserts    = [L + c + R for L, R in splits for c in letters]
    return set(deletes + transposes + replaces + inserts)

def edits2(word): 
    "All edits that are two edits away from `word`."
    return (e2 for e1 in edits1(word) for e2 in edits1(e1))

In [ ]:
### UTILITY FUNCTIONS
def inserts1(word):
    splits = [(word[:i], word[i:])    for i in range(len(word) + 1)]
    return [L + c + R               for L, R in splits for c in letters]

def inserts2(word):
    splits     = [(word[:i], word[i:])    for i in range(len(word) + 1)]
    return [i2 for i1 in inserts1(word) for i2 in inserts1(i1)]

def transposes(word):
    splits     = [(word[:i], word[i:])    for i in range(len(word) + 1)]
    return [L + R[1] + R[0] + R[2:] for L, R in splits if len(R)>1]

def replaces(word):
    splits     = [(word[:i], word[i:])    for i in range(len(word) + 1)]
    return [L + c + R[1:]           for L, R in splits if R for c in letters]
  
def numberofdupes(string, idx):
    """return the number of times in a row the letter at index idx is duplicated"""
    # "abccdefgh", 2  returns 1
    initial_idx = idx
    last = string[idx]
    while idx+1 < len(string) and string[idx+1] == last:
        idx += 1
    return idx-initial_idx

### POSSIBILITIES ANALYSIS
def edits3(word):
    "All edits that are two edits away from `word`."
    return (e3 for e1 in edits1(word) for e2 in edits1(e1) for e3 in edits1(e2))
  

In [ ]:
def spelltest(tests, verbose=False):
  "Run correction(wrong) on all (right, wrong) pairs; report results."
  import time
  start = time.clock()
  good, unknown = 0, 0
  n = len(tests)
  for right, wrong in tests:
      w = correction(wrong)
      good += (w == right)
      if w != right:
          unknown += (right not in WORDS)
          if verbose:
              print('correction({}) => {} ({}); expected {} ({})'
                    .format(wrong, w, WORDS[w], right, WORDS[right]))
  dt = time.clock() - start
  print('{:.0%} of {} correct ({:.0%} unknown) at {:.0f} words per second '
        .format(good / n, n, unknown / n, n / dt))
    
def Testset(lines):
  "Parse 'right: wrong1 wrong2' lines into [('right', 'wrong1'), ('right', 'wrong2')] pairs."
  return [(right, wrong)
          for (right, wrongs) in (line.split(':') for line in lines)
          for wrong in wrongs.split()]
spelltest(Testset(open('/content/drive/MyDrive/graduate/nlp/week1/spell-testset1.txt'))) # Development set
spelltest(Testset(open('/content/drive/MyDrive/graduate/nlp/week1/spell-testset2.txt'))) # Final test set

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: DeprecationWarning: time.clock has been deprecated in Python 3.3 and will be removed from Python 3.8: use time.perf_counter or time.process_time instead
  after removing the cwd from sys.path.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:15: DeprecationWarning: time.clock has been deprecated in Python 3.3 and will be removed from Python 3.8: use time.perf_counter or time.process_time instead
  from ipykernel import kernelapp as app


80% of 270 correct (6% unknown) at 29 words per second 
72% of 400 correct (11% unknown) at 25 words per second 
